In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn import ensemble, tree, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
#Read feature csv files and concate as single dataframe

path_f = r'C:\Users\Kelly\Desktop\iotml\FRS_Safety\Feature'  # use your path
all_feature_files = glob.glob(path_f + "/*.csv")

li = []

for filename in all_feature_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
df = pd.concat(li, axis=0, ignore_index=True)


In [ ]:
# Had used only one csv for the iot assignment due to the amount of time needed to build the model for 16million records 
# Read label csv file

#df = pd.read_csv('FRS_Safety/Feature/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv', index_col=None, header=0)

In [ ]:
#Check for any null values
df.isnull().sum()

In [ ]:
# Read label csv file
labels = pd.read_csv('FRS_Safety/Label/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv', index_col=None, header=0)

In [ ]:
# Sort the values based on the bookingids
labels = labels.sort_values(["bookingID"], ascending = [True])

In [ ]:
labels = labels.reset_index(drop = True)

In [ ]:
#Check what the labels dataframe contains
labels

In [ ]:
#Try to drop any duplicates from the labels to ensure there are no repeats
labels.drop_duplicates(subset="bookingID", keep="last", inplace=True)

In [ ]:
labels

In [ ]:
#Merge the labels and features dataframe to be used to build the model
result = pd.merge(df, labels, on = "bookingID", how = "inner")
result = result.sort_values(["bookingID"], ascending = [True])
result = result.reset_index(drop = True)

In [ ]:
#Checking the result dataframe 
result

In [ ]:
# seperation of target and feature variables
target = result["label"]
features = result.drop("label", axis = 1)

In [ ]:
target

In [ ]:
features

In [ ]:
# standardize the input feature 
sc = StandardScaler()
features = sc.fit_transform(features)

In [ ]:
features

In [ ]:
#Splitting the data to build the model and test
#
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.25, random_state = 42)

In [ ]:
values = [{"hidden_layer__n_hidden": [20, 100, 150, 200, 205],
         "hidden_layer__activation_func": ["tanh", "sigmoid", "gaussian"]}]

In [ ]:
#Used extreme machnine learning with the above functions.
elm = GridSearchCV(GenELMClassifier(), values, cv = 10)


In [ ]:
#Creating the model
elm.fit(X_train, y_train)

In [ ]:
#Testing the model created
y_pred = elm.predict(X_test)

In [ ]:
#Looking at the accuracy of the model
print("Accuracy of the model: " + str(elm.best_score_))